In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score, median_absolute_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor, VotingRegressor, BaggingRegressor

In [2]:
target, nn_batch_size = 'Item_Outlet_Sales', 64
training = pd.read_csv('training.csv')
testing = pd.read_csv('testing.csv')
x_train, x_test, y_train, y_test = training.drop(columns=[target]), testing.drop(columns=[target]), training[target], testing[target]
len(x_train.columns)

35